#### **Install Necessary Libraries**

In [1]:
!pip install torch
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-<YOUR-TORCH-VERSION>+cpu.html
!pip install torch torchvision torchaudio torch_geometric
!pip install torch-geometric torch-sparse torch-scatter -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.4 MB/s eta 0:00:00
/bin/bash: line 1: YOUR-TORCH-VERSION: No such file or directory
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.0/544.0 kB 30.0 MB/s eta 0:00:00


#### **Import the required Libraries**

In [2]:
import torch                                      # The main PyTorch library for tensor computation.
import torch.nn as nn                             # Provides classes and functions for building neural networks.
import torch.optim as optim                       # Contains various optimization algorithms for training neural networks.
from torch_geometric.nn import GCNConv            # A Graph Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import GATConv            # A Graph Attention Network (GAT) Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import SAGEConv           # A GraphSAGE Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import TransformerConv    # A Transformer Convolutional Layer from PyTorch Geometric.
import torch.nn.functional as F                   # Contains various functions for building neural networks (e.g., activation functions).
from torch_geometric.data import Data             # A class for graph data in PyTorch Geometric.
from torch.optim import Adam                      # Adam optimization algorithm for training neural networks.
from torch.nn.functional import cross_entropy     # Cross-entropy loss function for classification tasks.
from torch_geometric.loader import NeighborLoader # Loads graph data with neighbor sampling for efficient training.
from torch_geometric.loader import DataLoader     # A DataLoader for graph data in PyTorch Geometric.
import networkx as nx                             # Library for graph and network analysis.
from torch_geometric.utils import from_networkx   # Converts a NetworkX graph to PyTorch Geometric format.
from sklearn.preprocessing import StandardScaler  # Standardizes features by removing the mean and scaling to unit variance.
from sklearn.model_selection import train_test_split # Splits datasets into training and testing subsets.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score # Metrics for evaluating models.
from tqdm import tqdm                             # For progress tracking in loops.
import networkx as nx                             # Library for creating and analyzing graph data structures.
import pickle                                     # For saving and loading Python objects (e.g., models, data).
import os                                         # For file and directory operations.
import itertools                                  # For working with iterators and combinations.
from itertools import product                     # For generating the Cartesian product of input iterables.
import pandas as pd                               # For data manipulation and analysis.
import numpy as np                                # For numerical computations.
import seaborn as sns                             # For data visualization.
import matplotlib.pyplot as plt                   # For creating visualizations.
# To ignore warnings
import warnings                                   # Handles Python warnings.
warnings.filterwarnings("ignore")                # Suppresses all warnings.

#### **Load Edge-Centric Graph**

In [3]:
# Define the path to the saved edge-centric graph
edge_centric_graph_path = '/content/drive/MyDrive/GraphFeatures/EdgeCentricGraph.pt'

# Load the edge-centric graph
edge_centric_data = torch.load(edge_centric_graph_path)
print(f"Edge-centric graph loaded with {edge_centric_data.num_nodes} nodes and {edge_centric_data.num_edges} edges.")

# Validate that all required attributes are present
assert edge_centric_data.x is not None, "Node features are missing!"
assert edge_centric_data.edge_index is not None, "Edge index is missing!"
assert edge_centric_data.y is not None, "Target labels are missing!"

print(f"Node feature shape: {edge_centric_data.x.shape}")
print(f"Edge index shape: {edge_centric_data.edge_index.shape}")
print(f"Target labels shape: {edge_centric_data.y.shape}")

Edge-centric graph loaded with 2349356 nodes and 123219806 edges.
Node feature shape: torch.Size([2349356, 202])
Edge index shape: torch.Size([2, 123219806])
Target labels shape: torch.Size([2349356])


##### **Scale the Node Features**

In [4]:
# Scale node features using StandardScaler
scaler = StandardScaler()
edge_centric_data.x = torch.tensor(
    scaler.fit_transform(edge_centric_data.x.cpu().numpy()),
    dtype=torch.float
).to(edge_centric_data.x.device)  # Ensure the scaled features are on the correct device
print("Node features scaled successfully.")

Node features scaled successfully.


##### **Train-Validation-Test Split**

In [5]:
# Split the nodes into train, validation, and test sets
num_nodes = edge_centric_data.num_nodes
train_size = int(0.6 * num_nodes)
val_size = int(0.2 * num_nodes)
test_size = num_nodes - train_size - val_size

# Generate random permutations for shuffling the nodes
perm = torch.randperm(num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size + val_size]] = True
test_mask[perm[train_size + val_size:]] = True

# Assign the masks to the graph
edge_centric_data.train_mask = train_mask
edge_centric_data.val_mask = val_mask
edge_centric_data.test_mask = test_mask
print("Train, validation, and test masks created successfully.")

Train, validation, and test masks created successfully.


#### **Edge-Centric Graph with Graph Convolution Network Model**

##### **Define the GCN Model**

In [ ]:
# Define the Model
class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


##### **Initialize & Train Model**

In [ ]:
# Define model parameters
input_dim = edge_centric_data.x.size(1)  # Number of features per node
hidden_dim = 32
output_dim = edge_centric_data.y.max().item() + 1  # Number of classes

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNModel(input_dim, hidden_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
edge_centric_data = edge_centric_data.to(device)

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(edge_centric_data.x, edge_centric_data.edge_index)
    loss = cross_entropy(out[edge_centric_data.train_mask], edge_centric_data.y[edge_centric_data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = criterion(out[edge_centric_data.val_mask], edge_centric_data.y[edge_centric_data.val_mask])
        val_accuracy = (
            out[edge_centric_data.val_mask].argmax(dim=1) == edge_centric_data.y[edge_centric_data.val_mask]
        ).float().mean().item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

Epoch 1/50, Loss: 0.8140, Val Loss: 0.8134, Val Accuracy: 0.4770
Epoch 2/50, Loss: 0.6534, Val Loss: 0.6539, Val Accuracy: 0.5580
Epoch 3/50, Loss: 0.6672, Val Loss: 0.6676, Val Accuracy: 0.5873
Epoch 4/50, Loss: 0.6178, Val Loss: 0.6174, Val Accuracy: 0.7038
Epoch 5/50, Loss: 0.5783, Val Loss: 0.5774, Val Accuracy: 0.7286
Epoch 6/50, Loss: 0.5784, Val Loss: 0.5774, Val Accuracy: 0.7074
Epoch 7/50, Loss: 0.5739, Val Loss: 0.5731, Val Accuracy: 0.7076
Epoch 8/50, Loss: 0.5432, Val Loss: 0.5428, Val Accuracy: 0.7226
Epoch 9/50, Loss: 0.5070, Val Loss: 0.5070, Val Accuracy: 0.7481
Epoch 10/50, Loss: 0.4893, Val Loss: 0.4894, Val Accuracy: 0.7653
Epoch 11/50, Loss: 0.4935, Val Loss: 0.4936, Val Accuracy: 0.7642
Epoch 12/50, Loss: 0.5036, Val Loss: 0.5037, Val Accuracy: 0.7580
Epoch 13/50, Loss: 0.5033, Val Loss: 0.5034, Val Accuracy: 0.7593
Epoch 14/50, Loss: 0.4900, Val Loss: 0.4901, Val Accuracy: 0.7687
Epoch 15/50, Loss: 0.4727, Val Loss: 0.4729, Val Accuracy: 0.7796
Epoch 16/50, Loss: 

##### **Evaluate Model**

In [ ]:
# Evaluate Model
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out.argmax(dim=1)  # Predicted labels

        # Filter for test nodes
        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = preds[data.test_mask].cpu().numpy()

        # Compute metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        auc = roc_auc_score(y_true, out[data.test_mask][:, 1].cpu().numpy())

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print(f"AUC-ROC: {auc:.4f}")

# Evaluate the trained model
evaluate_model(model, edge_centric_data)

Accuracy: 0.9345
Precision: 0.9326
Recall: 0.9757
F1-score: 0.9537
AUC-ROC: 0.9550


#### **Edge-Centric Graph with GAT Model**

##### **Define the Model**

In [6]:
# Define the GAT model
class GATModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=1):
        super(GATModel, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
        self.gat2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        # First GAT layer with activation
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        # Second GAT layer
        x = self.gat2(x, edge_index)
        return F.log_softmax(x, dim=1)

##### **Initializing and Training the Model**

In [ ]:
# Model parameters
input_dim = edge_centric_data.x.size(1)  # Number of features per node
hidden_dim = 32
output_dim = edge_centric_data.y.unique().size(0)  # Number of unique classes
heads = 4  # Multi-head attention

# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATModel(input_dim, hidden_dim, output_dim, heads=heads).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# Move data to the device
edge_centric_data = edge_centric_data.to(device)

# Training loop parameters
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(edge_centric_data.x, edge_centric_data.edge_index)

    # Compute loss on training nodes
    loss = criterion(out[edge_centric_data.train_mask], edge_centric_data.y[edge_centric_data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = criterion(out[edge_centric_data.val_mask], edge_centric_data.y[edge_centric_data.val_mask])
        val_accuracy = (
            out[edge_centric_data.val_mask].argmax(dim=1) == edge_centric_data.y[edge_centric_data.val_mask]
        ).float().mean().item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

Epoch 1/50, Loss: 2.0183, Val Loss: 2.0428, Val Accuracy: 0.5866
Epoch 2/50, Loss: 1.7059, Val Loss: 1.7017, Val Accuracy: 0.5582
Epoch 3/50, Loss: 1.0894, Val Loss: 1.0921, Val Accuracy: 0.6245
Epoch 4/50, Loss: 0.7195, Val Loss: 0.7222, Val Accuracy: 0.6907
Epoch 5/50, Loss: 0.6090, Val Loss: 0.6102, Val Accuracy: 0.7169
Epoch 6/50, Loss: 0.5952, Val Loss: 0.5954, Val Accuracy: 0.7118
Epoch 7/50, Loss: 0.5934, Val Loss: 0.5925, Val Accuracy: 0.7136
Epoch 8/50, Loss: 0.5751, Val Loss: 0.5772, Val Accuracy: 0.7359
Epoch 9/50, Loss: 0.5488, Val Loss: 0.5500, Val Accuracy: 0.7468
Epoch 10/50, Loss: 0.5262, Val Loss: 0.5270, Val Accuracy: 0.7543
Epoch 11/50, Loss: 0.4977, Val Loss: 0.4993, Val Accuracy: 0.7780
Epoch 12/50, Loss: 0.4880, Val Loss: 0.4895, Val Accuracy: 0.7835
Epoch 13/50, Loss: 0.4926, Val Loss: 0.4916, Val Accuracy: 0.7752
Epoch 14/50, Loss: 0.4924, Val Loss: 0.4920, Val Accuracy: 0.7749
Epoch 15/50, Loss: 0.4783, Val Loss: 0.4787, Val Accuracy: 0.7889
Epoch 16/50, Loss: 

##### **Evaluate Model**

In [ ]:
# Evaluation function
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out.argmax(dim=1)

        # Filter for test nodes
        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = preds[data.test_mask].cpu().numpy()

        # Metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        auc = roc_auc_score(
            y_true, out[data.test_mask][:, 1].cpu().numpy()
        ) if len(data.y.unique()) == 2 else "Not applicable (multi-class)"

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print(f"AUC-ROC: {auc:.4f}")


# Evaluate the trained model
evaluate_model(model, edge_centric_data)

Accuracy: 0.9572
Precision: 0.9843
Recall: 0.9533
F1-score: 0.9685
AUC-ROC: 0.9830


#### **Edge-Centric Graph with GraphSAGE Model**

##### **Define Model**

In [7]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


##### **Initialize and Train Model**

In [8]:
# Model parameters
input_dim = edge_centric_data.x.size(1)  # Number of node features
hidden_dim = 32
output_dim = 2  # Binary classification (isFraud)
learning_rate = 0.01
weight_decay = 5e-4
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(input_dim, hidden_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = torch.nn.NLLLoss()

# Move data to device
edge_centric_data = edge_centric_data.to(device)

# Train-test-validation masks
train_mask = edge_centric_data.train_mask
val_mask = edge_centric_data.val_mask
test_mask = edge_centric_data.test_mask

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(edge_centric_data.x, edge_centric_data.edge_index)
    loss = criterion(out[train_mask], edge_centric_data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        val_loss = criterion(out[val_mask], edge_centric_data.y[val_mask])
        val_acc = (out[val_mask].argmax(dim=1) == edge_centric_data.y[val_mask]).float().mean()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_acc.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Loss: 0.7115, Val Loss: 0.7115, Val Accuracy: 0.4616
Epoch 2/50, Loss: 0.5850, Val Loss: 0.5858, Val Accuracy: 0.6529
Epoch 3/50, Loss: 0.5447, Val Loss: 0.5460, Val Accuracy: 0.7626
Epoch 4/50, Loss: 0.5267, Val Loss: 0.5279, Val Accuracy: 0.7559
Epoch 5/50, Loss: 0.4814, Val Loss: 0.4821, Val Accuracy: 0.7778
Epoch 6/50, Loss: 0.4636, Val Loss: 0.4638, Val Accuracy: 0.7822
Epoch 7/50, Loss: 0.4673, Val Loss: 0.4676, Val Accuracy: 0.7783
Epoch 8/50, Loss: 0.4440, Val Loss: 0.4446, Val Accuracy: 0.7896
Epoch 9/50, Loss: 0.4166, Val Loss: 0.4175, Val Accuracy: 0.8102
Epoch 10/50, Loss: 0.4087, Val Loss: 0.4098, Val Accuracy: 0.8177
Epoch 11/50, Loss: 0.4032, Val Loss: 0.4044, Val Accuracy: 0.8184
Epoch 12/50, Loss: 0.3806, Val Loss: 0.3817, Val Accuracy: 0.8351
Epoch 13/50, Loss: 0.3600, Val Loss: 0.3609, Val Accuracy: 0.8547
Epoch 14/50, Loss: 0.3524, Val Loss: 0.3531, Val Accuracy: 0.8599
Epoch 15/50, Loss: 0.3371, Val Loss: 0.3377, Val Accuracy: 0.8681
Epoch 16/50, Loss: 

##### **Evaluate Model**

In [9]:
def evaluate_model(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out[mask].argmax(dim=1).cpu().numpy()
        labels = data.y[mask].cpu().numpy()

    # Compute metrics
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, zero_division=0)
    recall = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)
    auc = roc_auc_score(labels, out[mask][:, 1].cpu().numpy())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

# Evaluate on test set
evaluate_model(model, edge_centric_data, test_mask)

Accuracy: 0.9994
Precision: 0.9994
Recall: 0.9998
F1-Score: 0.9996
AUC-ROC: 1.0000


#### **Edge-Centric Graph with Graphomer Transformer Model**

##### **Define Model**

In [10]:
class Graphomer(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, dropout=0.1):
        super(Graphomer, self).__init__()
        self.conv1 = TransformerConv(input_dim, hidden_dim, heads=num_heads, dropout=dropout)
        self.conv2 = TransformerConv(hidden_dim * num_heads, output_dim, heads=1, dropout=dropout)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


##### **Initialize and Train Model**

In [12]:
# Model parameters
input_dim = edge_centric_data.x.size(1)  # Number of node features
hidden_dim = 32
output_dim = 2  # Binary classification (isFraud)
num_heads = 2  # Multi-head attention
dropout = 0.1
learning_rate = 0.005
weight_decay = 5e-4

# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graphomer(input_dim, hidden_dim, output_dim, num_heads, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = torch.nn.NLLLoss()

# Move data to device
edge_centric_data = edge_centric_data.to(device)

# Train-test-validation masks
train_mask = edge_centric_data.train_mask
val_mask = edge_centric_data.val_mask
test_mask = edge_centric_data.test_mask

# Training Loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(edge_centric_data.x, edge_centric_data.edge_index)
    train_loss = criterion(out[train_mask], edge_centric_data.y[train_mask])
    train_loss.backward()
    optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        val_loss = criterion(out[val_mask], edge_centric_data.y[val_mask])
        val_acc = (out[val_mask].argmax(dim=1) == edge_centric_data.y[val_mask]).float().mean()

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_acc.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stopping_counter = 0
    else:
        stopping_counter += 1
        if stopping_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Train Loss: 0.6829, Val Loss: 0.6832, Val Accuracy: 0.6230
Epoch 2/50, Train Loss: 0.5398, Val Loss: 0.5412, Val Accuracy: 0.7491
Epoch 3/50, Train Loss: 0.5112, Val Loss: 0.5122, Val Accuracy: 0.7633
Epoch 4/50, Train Loss: 0.4866, Val Loss: 0.4870, Val Accuracy: 0.7762
Epoch 5/50, Train Loss: 0.4592, Val Loss: 0.4598, Val Accuracy: 0.7882
Epoch 6/50, Train Loss: 0.4439, Val Loss: 0.4446, Val Accuracy: 0.7952
Epoch 7/50, Train Loss: 0.4241, Val Loss: 0.4248, Val Accuracy: 0.8107
Epoch 8/50, Train Loss: 0.4028, Val Loss: 0.4037, Val Accuracy: 0.8260
Epoch 9/50, Train Loss: 0.3801, Val Loss: 0.3811, Val Accuracy: 0.8400
Epoch 10/50, Train Loss: 0.3595, Val Loss: 0.3607, Val Accuracy: 0.8554
Epoch 11/50, Train Loss: 0.3399, Val Loss: 0.3410, Val Accuracy: 0.8706
Epoch 12/50, Train Loss: 0.3139, Val Loss: 0.3149, Val Accuracy: 0.8867
Epoch 13/50, Train Loss: 0.2844, Val Loss: 0.2853, Val Accuracy: 0.9026
Epoch 14/50, Train Loss: 0.2561, Val Loss: 0.2569, Val Accuracy: 0.9178
E

##### **Evaluate Model Performance**

In [13]:
def evaluate_model(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out[mask].argmax(dim=1).cpu().numpy()
        labels = data.y[mask].cpu().numpy()

    # Compute metrics
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, zero_division=0)
    recall = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)
    auc = roc_auc_score(labels, out[mask][:, 1].cpu().numpy())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

# Evaluate on test set
evaluate_model(model, edge_centric_data, test_mask)

Accuracy: 0.9994
Precision: 0.9994
Recall: 0.9997
F1-Score: 0.9996
AUC-ROC: 1.0000
